https://blog.csdn.net/weixin_42608414/article/details/135266719

In [3]:
from llama_index.readers.web import TrafilaturaWebReader
 
docs = TrafilaturaWebReader().load_data(
         ["https://baike.baidu.com/item/ChatGPT/62446358",
          "https://baike.baidu.com/item/恐龙/139019"]
)

In [4]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode
 
#创建文档切割器
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7faa54266110>, id_func=<function default_id_func at 0x7faa60ecd000>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [5]:
base_nodes = node_parser.get_nodes_from_documents(docs)
 
len(base_nodes)

40

In [6]:
base_nodes[10].text

'收藏\n查看我的收藏\n0有用+1\n恐龙（英文名：Dinosaur），是指三角龙、现代鸟类和梁龙最近的共同祖先及其所有后代。 [1]\n矫健的四肢、长长的尾巴和庞大的身躯是部分非鸟恐龙的写照。它们主要栖息于湖岸平原（或海岸平原）上的森林地或开阔地带。\n1915年，美国加利福尼亚大学地质学家乔治·D·劳德伯克博士和R·R·莫斯博士在自贡市发现恐龙化石。 [19] [22]2022年3月，云南大学生命科学院的脊椎动物研究团队发现科氏玉溪龙，是在亚洲发现的迄今最早的有甲类恐龙。 [16]\n- TA说\n找一只恐龙，有多难？2023-04-17 21:19\n每一只与人类“相遇”的恐龙都极其幸运。它们要躲避各种分解、侵蚀，经历埋藏、搬运、石化，才能以化石的形态穿越亿万年的岁月保存下来。而人类，则要经过大海捞针般的搜寻，加上亿点点时间，亿点点运气，才能最终与恐龙“相见”。跟随我们的脚步，穿越时空，一起见证这场人类与恐龙的“双向奔赴”！详情\n内容来自\n- 纲\n- 蜥形纲\n- 亚 纲\n- 双孔亚纲\n- 目\n- 鸟臀目（Ornithischuia)、蜥臀目(Saurischia)\n- 次亚纲\n- 主龙型次亚纲\n- 超 目\n- 恐龙总目\n- 生存时间\n- 中晚三叠世至白垩纪末\n- 词 源\n- 日本古生物学家译为恐竜\n“恐龙”一词在西方指恐蜥（恐怖的蜥蜴），日本的古生物学家译为“恐竜”，后来流行于中国。古希腊语蜥蜴（Saurosc），特指一些巨大的有四肢有尾或兼有翼的类似蜥蜴的爬虫。 [2]\n人类发现恐龙化石的历史由来已久。早在曼特尔夫妇发现禽龙之前，欧洲人就知道地下埋藏有许多奇形怪状的巨大骨骼化石。但是，当时人们并不知道它们的确切归属，因此一直误认为是“巨人的遗骸” [3]。直到曼特尔夫妇发现了禽龙并与鬣蜥进行对比，科学界才初步确定了这是一种类似于蜥蜴的、早已灭绝的爬行动物。'

In [7]:
from llama_index.embeddings import resolve_embed_model
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI
# from llama_index.llms import Gemini
 
 
#创建BAAI的embedding
embed_model = resolve_embed_model("local:BAAI/bge-small-zh-v1.5")
 
#创建OpenAI的llm
llm = OpenAI(model="gpt-3.5-turbo",system_prompt="总是用中文回答。")
 
#创建谷歌gemini的llm
#llm = Gemini()
 
#创建service_context 
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [8]:
#创建index
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
#创建检索器
base_retriever = base_index.as_retriever(similarity_top_k=2)
 
#检索相关文档
retrievals = base_retriever.retrieve(
    "恐龙是冷血动物吗？"
)

In [9]:
from llama_index.response.notebook_utils import display_source_node
 
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** 7d73fdfd-4358-4863-95f0-74adbe1ca385<br>**Similarity:** 0.663452407287916<br>**Text:** 自从1970年以来，许多研究报告指出现代鸟类极可能是兽脚亚目恐龙的直系后代，近年来的研究都倾向于把鸟类直接归入虚骨龙类（包括霸王龙、窃蛋龙、恐爪龙等）。鳄鱼则是另一群恐龙的现代近亲，但两者关系较非鸟恐龙与鸟类远。非鸟恐龙、鸟类、鳄鱼都属于初龙类演化支，该演化支首次出现于晚二叠纪，并在中三叠纪成为优势动物群。 [12]
哺乳动物起源于爬行动物，它们的前身是“似哺乳类的爬行动物”，即兽孔目，早期则是“似爬行类的哺乳动物”，即哺乳型动物。 [12]
中生代的爬行动物，大部分在中生代的末期灭绝了；一部分适应了变化的环境被保留下来，即现存的爬行动物（如龟鳖类、蛇类、鳄类等）；还有一部分沿着不同的进化方向，进化成了现今的鸟类和哺乳类。 [12]
恐龙是介于冷血和温血之间的动物
2014年6月，有关恐龙究竟是像鸟类和哺乳动物一样的温血动物，还是类似爬行动物、鱼类和两栖动物的冷血动物的问题终于有了答案——恐龙其实是介于冷血和温血之间的动物。 [12]
“我们的结果显示恐龙所具有的生长速率和新陈代谢速率，既不是冷血生物体也不是温血生物体所具有的特征。它们既不像哺乳动物或者鸟类，也不像爬行动物或者鱼类，而是介于现代冷血动物和温血动物之间。简言之，它们的生理机能在现代社会并不常见。”美国亚利桑那大学进化生物学家和生态学家布莱恩·恩奎斯特说。
墨西哥生物学家表示，正是这种中等程度的新陈代谢使得恐龙可以长得比任何哺乳动物都要大。温血动物需要大量进食，因此它们频繁猎捕和咀嚼植物。“很难想象霸王龙大小的狮子能够吃饱以存活下来。 [12]
2022年3月16日，云南大学生命科学院的脊椎动物研究团队，在国际生物学著名期刊《eLife》上报道了一件基干有甲类恐龙的化石骨架，依据其头骨、脊椎和肢骨的自近裔特征，认为是一新属种，取名为科氏玉溪龙，是在亚洲发现的迄今最早的有甲类恐龙。<br>

**Node ID:** e7870cfe-2e54-4e4d-b456-f069a81e0325<br>**Similarity:** 0.6466952097539082<br>**Text:** 从食肉恐龙远远少于食草恐龙来看，这一点也是合理的。另外，还有一些身体较小的恐龙，它们身上覆盖着一层羽毛或毛发，这也是为了防止体温散失。其它方面，如骨骼的研究，也初步表明一些恐龙是温血动物。温血恐龙的说法一提出，就受到强烈抨击，但到底结论如何，还难下定论。 [12]
有些人认为恐龙是温血性动物，因此可能禁不起白垩纪晚期的寒冷气候而导致无法存活。因为即使恐龙是温血性，体温仍然不高，可能和现生树懒的体温差不多，而要维持这样的体温，也只能生存在热带气候区。同时恐龙的呼吸器官并不完善，不能充分补给氧。温血动物和冷血动物不一样的地方，就是如果体温降到一定的范围之下，就要消耗体能以提高体温，身体也就很快地变得虚弱。它们过于庞大的体驱，不能进入洞中避寒，所以如果寒冷的日子持续几天，可能就会因为耗尽体力而遭到冻死的命运。但是，这种学说有一个疑点，那就是恐龙不都是那么庞大的，也不一定都不能躲进洞里避难，所以这种学说也有不完善的地方。 [12]
哺乳进化说（偷吃说）
在中生代后半，已有哺乳类的祖先生存。据化石的记录，当时的哺乳类体型甚小，数量也有限，直到白垩纪的后期，数量才开始急速增加。推测它们属于以昆虫等为主食的杂食性，这些小型哺乳类发现恐龙卵之后，即不断取而食之，最终导致恐龙的生育危机，导致恐龙灭绝。 [12]
骤变理论说
根据深海地质钻探得到的资料，一些科学家认为在6500万年前的地球上的气候发生了异常的变化，温度忽然升高。这种变化使恐龙等散热能力较弱的变温动物无法很好地适应环境，引起其身体中的内分泌系统紊乱，尤其是造成雄性个体的生殖系统严重损坏。结果，恐龙无法繁殖后代，从而走向了最终的绝灭。 [12]
还有一种理论，虽然同样是认为气候骤变引起恐龙绝灭，但是推测的过程却不一样。这一派学者认为，在距今大约7000万年前，北冰洋与其它大洋之间被陆地完全隔开，并在最后的日子里，那咸咸的海水因各种因素的作用渐渐地变成了淡水。<br>

In [10]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c,chunk_overlap=0) for c in sub_chunk_sizes
]
 
sub_node_parsers

[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7fa940282740>, id_func=<function default_id_func at 0x7faa60ecd000>, chunk_size=128, chunk_overlap=0, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'),
 SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7fa940282230>, id_func=<function default_id_func at 0x7faa60ecd000>, chunk_size=256, chunk_overlap=0, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'),
 SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7fa940281fc0>, id_func=<function default_id_func at 0x7faa60ecd000>, chunk_size=512, chunk_overlap=0, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]

In [11]:
all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)
 
    #添加父节点文档
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [12]:
all_nodes[:2]

[IndexNode(id_='548ce39b-1cc9-405f-935f-4532e72ea7d3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2b8b20ca-bc45-4197-aa7a-bae415bf4570', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6e13bcb8a6a41c561f2824499b58c40e8c1efbd6e9d14832b544b776b62a40e7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6de8ddc8-afa7-46e8-9593-426752da7170', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='cb8bb8cc77cdbb46881072a3094a8a2f2f79d4f4ce2e2a14b8ec90c5968476e7')}, hash='de3f2e59ad0c58d135bf954b1c2053453447d908c06ffad9ac717da35c1eae35', text='收藏\n查看我的收藏\n0有用+1\nChatGPT（全名：Chat Generative Pre-trained Transformer），是OpenAI [1]研发的一款聊天机器人程序 [12]，于2022年11月30日发布 [2-3]。', start_char_idx=0, end_char_idx=118, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='2b8b20ca-bc45-4197-aa7a-bae415bf4570'),
 IndexNod

In [ ]:
all_nodes_dict = {n.node_id: n for n in all_nodes}
 
#查看特定节点
all_nodes_dict['548ce39b-1cc9-405f-935f-4532e72ea7d3']

In [16]:
from llama_index.retrievers import RecursiveRetriever
 
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)
 
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)
 
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [16]:
nodes = retriever_chunk.retrieve(
   "恐龙是冷血动物吗？"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: 恐龙是冷血动物吗？
Retrieved node with id, entering: c9f365f3-2407-416a-b711-6746ecb52f1e
Retrieving with query id c9f365f3-2407-416a-b711-6746ecb52f1e: 恐龙是冷血动物吗？
Retrieved node with id, entering: 61ffba20-3814-4818-b33a-40b7a83012ae
Retrieving with query id 61ffba20-3814-4818-b33a-40b7a83012ae: 恐龙是冷血动物吗？


**Node ID:** c9f365f3-2407-416a-b711-6746ecb52f1e<br>**Similarity:** 0.7532021422717489<br>**Text:** [12]
火山爆发说
因为火山的爆发，二氧化碳大量喷出，造成地球急激的温室效应，使得植物死亡。而且，火山喷发使得盐素大量释出，臭氧层破裂，有害的紫外线照射地球表面，造成生物灭亡。但这个学说有一个前提，那就是火山大规模地爆发。 [12]
意大利著名物理学家安东尼奥·齐基基提出，恐龙大绝灭的原因很可能是大规模的海底火山爆发。齐基基教授认为，白垩纪末期，地球上在海洋底下发生了一系列大规模的火山爆发，从而影响了海水的热平衡，并进而引起了陆地气候的变化，因此影响了需要大量食物维持生存的恐龙等动物的生存。 [12]
造山运动说
在白垩纪末期发生的造山运动使得沼泽干涸，许多以沼泽为家的恐龙就无法再生活下去。因为气候变化，植物也改变了，食草性的恐龙不能适应新的食物，而相继灭绝。食草性恐龙的灭绝使肉食性恐龙也失去了依持，结果也灭绝了。此灭绝过程持续了1000万～2000万年。到了白垩纪末期，恐龙在地球上绝迹。 [12]
温血动物说
过去，所有的科学家都认为恐龙像其他爬行动物一样是冷血动物或变温动物，但是随着化石资料的不断增多，人们的认识也发生了变化，有人提出，有些恐龙可能是温血动物。首先，他们认为有些恐龙行动极为敏捷，也不是像蛇一样在地上爬行，而是靠两条后腿在地面上跑动，其速度可达每小时20至90多公里。这就需要有强壮的心脏并且维持较高的新陈代谢，这些显然冷血动物是做不到的。其次，恐龙的食量都相当大，据推测，一头30吨重的蜥脚类恐龙，每天可能要吃掉近2吨食物，只有温血动物才需要这么多的能量。从食肉恐龙远远少于食草恐龙来看，这一点也是合理的。另外，还有一些身体较小的恐龙，它们身上覆盖着一层羽毛或毛发，这也是为了防止体温散失。其它方面，如骨骼的研究，也初步表明一些恐龙是温血动物。温血恐龙的说法一提出，就受到强烈抨击，但到底结论如何，还难下定论。<br>

**Node ID:** 61ffba20-3814-4818-b33a-40b7a83012ae<br>**Similarity:** 0.7531834820310072<br>**Text:** 自从1970年以来，许多研究报告指出现代鸟类极可能是兽脚亚目恐龙的直系后代，近年来的研究都倾向于把鸟类直接归入虚骨龙类（包括霸王龙、窃蛋龙、恐爪龙等）。鳄鱼则是另一群恐龙的现代近亲，但两者关系较非鸟恐龙与鸟类远。非鸟恐龙、鸟类、鳄鱼都属于初龙类演化支，该演化支首次出现于晚二叠纪，并在中三叠纪成为优势动物群。 [12]
哺乳动物起源于爬行动物，它们的前身是“似哺乳类的爬行动物”，即兽孔目，早期则是“似爬行类的哺乳动物”，即哺乳型动物。 [12]
中生代的爬行动物，大部分在中生代的末期灭绝了；一部分适应了变化的环境被保留下来，即现存的爬行动物（如龟鳖类、蛇类、鳄类等）；还有一部分沿着不同的进化方向，进化成了现今的鸟类和哺乳类。 [12]
恐龙是介于冷血和温血之间的动物
2014年6月，有关恐龙究竟是像鸟类和哺乳动物一样的温血动物，还是类似爬行动物、鱼类和两栖动物的冷血动物的问题终于有了答案——恐龙其实是介于冷血和温血之间的动物。 [12]
“我们的结果显示恐龙所具有的生长速率和新陈代谢速率，既不是冷血生物体也不是温血生物体所具有的特征。它们既不像哺乳动物或者鸟类，也不像爬行动物或者鱼类，而是介于现代冷血动物和温血动物之间。简言之，它们的生理机能在现代社会并不常见。”美国亚利桑那大学进化生物学家和生态学家布莱恩·恩奎斯特说。
墨西哥生物学家表示，正是这种中等程度的新陈代谢使得恐龙可以长得比任何哺乳动物都要大。温血动物需要大量进食，因此它们频繁猎捕和咀嚼植物。“很难想象霸王龙大小的狮子能够吃饱以存活下来。 [12]
2022年3月16日，云南大学生命科学院的脊椎动物研究团队，在国际生物学著名期刊《eLife》上报道了一件基干有甲类恐龙的化石骨架，依据其头骨、脊椎和肢骨的自近裔特征，认为是一新属种，取名为科氏玉溪龙，是在亚洲发现的迄今最早的有甲类恐龙。<br>

In [17]:
from llama_index.query_engine import RetrieverQueryEngine

In [19]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [20]:
#openai llm 的回答
response = query_engine_chunk.query(
    "恐龙是冷血动物吗？"
)
print(str(response))

Retrieving with query id None: 恐龙是冷血动物吗？
Retrieved node with id, entering: c9f365f3-2407-416a-b711-6746ecb52f1e
Retrieving with query id c9f365f3-2407-416a-b711-6746ecb52f1e: 恐龙是冷血动物吗？
Retrieved node with id, entering: 61ffba20-3814-4818-b33a-40b7a83012ae
Retrieving with query id 61ffba20-3814-4818-b33a-40b7a83012ae: 恐龙是冷血动物吗？
恐龙不是冷血动物。根据提供的信息，恐龙被认为是介于冷血和温血之间的动物。研究表明，恐龙具有中等程度的新陈代谢，不像典型的冷血动物或温血动物。这种中等程度的新陈代谢使得恐龙能够长得比任何哺乳动物都要大，并且不需要频繁进食。因此，恐龙被认为具有一种介于冷血和温血之间的生理机能。
